# Time-lapse Photography

This chapter builds upon the previous tutorial, capturing frames from the camera at regular intervals and saving them in the `/templates/pictures/` folder within the `ugv_jetson` directory.

## Preparation

Since the product automatically runs the main program at startup, which occupies the camera resource, this tutorial cannot be used in such situations. You need to terminate the main program or disable its automatic startup before restarting the robot.

It's worth noting that because the robot's main program uses multi-threading and is configured to run automatically at startup through crontab, the usual method sudo killall python typically doesn't work. Therefore, we'll introduce the method of disabling the automatic startup of the main program here.

### Terminate the Main Program

1. Click the "+" icon next to the tab for this page to open a new tab called "Launcher."
2. Click on "Terminal" under "Other" to open a terminal window.
3. Type bash into the terminal window and press Enter.
4. Now you can use the Bash Shell to control the robot.
5. Enter the command: `sudo killall -9 python`.

## Example

The following code block can be run directly:

1. Select the code block below.
2. Press Shift + Enter to run the code block.
3. Watch the real-time video window.
4. Press `STOP` to close the real-time video and release the camera resources.

### If you cannot see the real-time camera feed when running:

- Click on Kernel -> Shut down all kernels above.
- Close the current section tab and open it again.
- Click `STOP` to release the camera resources, then run the code block again.
- Reboot the device.

### Differences from the Previous Chapter

You can adjust the value of `time_intervel` to change the interval between photos, measured in seconds.
The photos you capture will be stored in the `/ugv_jetson/templates/pictures/` folder.

In [ ]:
import cv2  # Import the OpenCV library for image processing
import numpy as np
from IPython.display import display, Image  # Import IPython display functionality
import ipywidgets as widgets  # Import the ipywidgets library for creating cloud interactive widgets
import threading  # Import the threading library for multithreading

import os, time  # Import the os and time libraries for file operations and time-related functionalities

# Change the interval time for taking photos here (in seconds)
time_intervel = 3  # Take a photo every 3 seconds

# Set the path for saving the images
# You can change the save path here
photo_path = '/home/ws/ugv_pt_rpi/static/'

# Create a toggle button as a stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,  # The initial state of the button is unselected
    description='Stop',  # Text displayed on the button
    disabled=False,  # The button is initially enabled
    button_style='danger',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',  # Tooltip displayed when hovering over the button
    icon='square'  # Button icon (FontAwesome name without the `fa-` prefix)
)


# Define a function for displaying the video stream and taking photos at regular intervals
# ================
def view(button):
    last_picture_time = time.time()  # Record the time of the last photo taken
    num_count = 0  # Initialize the photo counter

    camera = cv2.VideoCapture(-1) 
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    display_handle=display(None, display_id=True)  # Create a display handle for updating the displayed content
    
    i = 0
    while True:
        # frame = picam2.capture_array()
        _, frame = camera.read()

        # Take a photo every few seconds
        if time.time() - last_picture_time >= time_intervel:
            num_count = num_count + 1  # Update the photo counter
            photo_filename = f'{photo_path}photo_{num_count}.jpg'  # Define the file name for the photo
            cv2.imwrite(photo_filename, frame)  # Save the photo to the specified path
            last_picture_time = time.time()  # Update the time of the last photo taken
            print(f'{num_count} photos saved. new photo: {photo_filename}')  # Print information about the saved photo
            
        _, frame = cv2.imencode('.jpeg', frame)  # Encode the frame as JPEG format
        display_handle.update(Image(data=frame.tobytes()))  # Update the displayed image
        if stopButton.value==True:  # Check if the stop button is pressed
            # picam2.close()
            cv2.release() # If yes, close the camera
            display_handle.update(None)

            
# Display the stop button and start the video stream display thread
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()